In [2]:
import os
import yaml

In [3]:
src_lang = "en"
tgt_lang = "de"
root = "/data/user_data/siqiouya/dataset/must-c-v1.0/{}-{}/data/tst-COMMON/txt".format(src_lang, tgt_lang)
wav_root = "/data/user_data/siqiouya/dataset/must-c-v1.0/{}-{}/data/tst-COMMON/wav".format(src_lang, tgt_lang)
manifest_root = "/data/user_data/siqiouya/dataset/must-c-v1.0/{}-{}/".format(src_lang, tgt_lang)

In [4]:
!head $root/tst-COMMON.en

Back in New York, I am the head of development for a non-profit called Robin Hood.
When I'm not fighting poverty, I'm fighting fires as the assistant captain of a volunteer fire company.
Now in our town, where the volunteers supplement a highly skilled career staff, you have to get to the fire scene pretty early to get in on any action.
I remember my first fire.
I was the second volunteer on the scene, so there was a pretty good chance I was going to get in.
But still it was a real footrace against the other volunteers to get to the captain in charge to find out what our assignments would be.
When I found the captain, he was having a very engaging conversation with the homeowner, who was surely having one of the worst days of her life.
Here it was, the middle of the night, she was standing outside in the pouring rain, under an umbrella, in her pajamas, barefoot, while her house was in flames.
The other volunteer who had arrived just before me — let's call him Lex Luther — (Laughter) go

In [5]:
with open(os.path.join(root, "tst-COMMON.yaml"), "r") as r:
    metas = yaml.load(r, yaml.Loader)
with open(os.path.join(root, "tst-COMMON.{}".format(tgt_lang)), "r") as r:
    refs = r.read().strip().split("\n")
with open(os.path.join(root, "tst-COMMON.{}".format(src_lang)), "r") as r:
    srcs = r.read().strip().split("\n")

In [6]:
metas[2]

{'duration': 8.1,
 'offset': 22.53,
 'speaker_id': 'spk.1096',
 'wav': 'ted_1096.wav'}

In [7]:
full_wavs = []
seg_wavs = []
full_refs = []
seg_refs = []
full_srcs = []
for i, (meta, ref, src) in enumerate(zip(metas, refs, srcs)):
    assert i == 0 or meta['wav'] != metas[i - 1]['wav'] or meta['offset'] > metas[i - 1]['offset']
    if i == 0 or meta['wav'] != metas[i - 1]['wav']:
        full_wavs.append(meta['wav'])
        seg_wavs.append("{} {}\n".format(meta['offset'], meta['duration']))
        full_refs.append(ref)
        seg_refs.append(ref)
        full_srcs.append(src)
    else:
        seg_wavs[-1] += "{} {}\n".format(meta['offset'], meta['duration'])
        full_refs[-1] += ' ' + ref
        seg_refs[-1] += '\n' + ref
        full_srcs[-1] += ' ' + src

In [8]:
with open(os.path.join(manifest_root, "tst-COMMON_full.source"), "w") as w_source, \
    open(os.path.join(manifest_root, "tst-COMMON_full_segmented.source"), "w") as w_source_s, \
    open(os.path.join(manifest_root, "tst-COMMON_full.target"), "w") as w_target, \
    open(os.path.join(manifest_root, "tst-COMMON_full_segmented.target"), "w") as w_target_s:
    for wav, wav_s, ref, ref_s in zip(full_wavs, seg_wavs, full_refs, seg_refs):
        w_source.write(os.path.join(wav_root, wav) + '\n')
        w_source_s.write(wav_s + '\n\n')
        w_target.write(ref + '\n')
        w_target_s.write(ref_s + '\n\n')

In [14]:
with open(os.path.join(manifest_root, "tst-COMMON_full.source.txt"), "w") as w_source:
    for src in full_srcs:
        w_source.write(src + '\n')

In [54]:
print(full_refs[0])

Zu Hause in New York, bin ich Chef der Entwicklungsabteilung einer gemeinnützigen Organisation namens Robin Hood. Wenn ich nicht die Armut bekämpfe, bekämpfe ich als Gehilfe eines Feuerwehr-Hauptmanns bei einem freiwilligen Löschzug das Feuer. Nun, in unserer Stadt, in der Freiwillige eine hochqualifizierte Berufsfeuerwehr unterstützten, muss man ziemlich früh an der Brandstelle sein, um mitmischen zu können. Ich erinnere mich an mein erstes Feuer. Ich war der zweite Freiwillige an der Brandstelle, ich hatte also recht gute Chancen hinein zu können. Aber es war immer noch ein Wettrennen gegen die anderen Freiwilligen um den verantwortlichen Hauptmann zu erreichen und herauszufinden was unsere Aufgaben sein würden. Als ich den Hauptmann fand, hatte er gerade in eine sehr ernste Unterhaltung mit der Hausbesitzerin, die sicherlich einen der schlimmsten Tage ihres Lebens hatte. Es war mitten in der Nacht und sie stand im Schlafanzug und barfuß unter einem Schirm draußen im strömenden Regen

In [46]:
import transformers
import torchaudio
tokenizer = transformers.AutoTokenizer.from_pretrained(
    "/data/user_data/siqiouya/runs/pretrained/llama-2-7b/hf",
    padding_side="right",    
    truncation=False,
    add_special_tokens=False
)

1069

In [52]:
for wav, ref in zip(full_wavs, full_refs):
    wav_path = os.path.join(wav_root, wav)
    wav_info = torchaudio.info(wav_path)
    wav_dur = wav_info.num_frames / wav_info.sample_rate
    n_token = len(tokenizer(ref)['input_ids'])
    print(n_token / wav_dur)

4.370257718472004
3.699776417034397
3.4563542460835515
5.299428039719741
5.048043494628684
4.082369117860166
4.321981241997159
4.584028203504232
4.9589603283173735
4.343389088012744
5.1406950682194905
5.6329289452725
2.941286979484372
3.9284438237883683
3.5791166846288522
4.940720835369553
4.745152098650434
4.7848183391003465
4.005777462018934
4.055033760387812
3.4629846612843997
4.170849336592179
3.894420787186417
4.56000912928657
5.024235960635608
4.701513729077847
4.5411693974268
